<a href="https://colab.research.google.com/github/ZubekKlaudia/Model-Uczenia-/blob/main/Poidzia%C5%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
!kaggle datasets download -d jessicali9530/stanford-dogs-dataset --unzip

Dataset URL: https://www.kaggle.com/datasets/jessicali9530/stanford-dogs-dataset
License(s): other
100% 748M/750M [00:26<00:00, 30.2MB/s]
100% 750M/750M [00:26<00:00, 29.9MB/s]


In [ ]:
import os

In [ ]:
data_dir = './'
input_dir = os.path.join(data_dir, './images/Images/')

In [ ]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Ustaw ścieżki
train_dir = './data/train'
val_dir = './data/val'
test_dir = './data/test'

# Utwórz katalogi na zbiory treningowy, walidacyjny i testowy
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Wczytaj wszystkie foldery klas (każdy folder to jedna rasa)
classes = os.listdir(input_dir)

# Ustaw proporcje podziału
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Dla każdej klasy utwórz odpowiedni podział
for class_name in classes:
    # Pomijaj ukryte pliki, jeśli takie są (np. .DS_Store na macOS)
    if class_name.startswith('.'):
        continue

    # Pobierz ścieżkę do folderu danej klasy
    class_path = os.path.join(input_dir, class_name)

    # Sprawdź, czy to faktycznie jest folder
    if not os.path.isdir(class_path):
        continue

    # Pobierz wszystkie obrazy dla danej klasy
    images = os.listdir(class_path)

    # Podziel na zbiory
    train_images, val_test_images = train_test_split(images, test_size=(1 - train_ratio))
    val_images, test_images = train_test_split(val_test_images, test_size=(test_ratio / (val_ratio + test_ratio)))

    # Utwórz katalogi dla danej klasy w zbiorach
    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    # Przenieś obrazy do odpowiednich katalogów
    for image in train_images:
        shutil.copy(os.path.join(class_path, image), os.path.join(train_dir, class_name, image))

    for image in val_images:
        shutil.copy(os.path.join(class_path, image), os.path.join(val_dir, class_name, image))

    for image in test_images:
        shutil.copy(os.path.join(class_path, image), os.path.join(test_dir, class_name, image))

print("Dane zostały podzielone na zbiory treningowy, walidacyjny i testowy.")


Dane zostały podzielone na zbiory treningowy, walidacyjny i testowy.


In [ ]:
import os

# Liczenie liczby zdjęć w każdym z katalogów (treningowy, walidacyjny, testowy)
train_images_count = sum([len(os.listdir(os.path.join(train_dir, class_name))) for class_name in os.listdir(train_dir)])
val_images_count = sum([len(os.listdir(os.path.join(val_dir, class_name))) for class_name in os.listdir(val_dir)])
test_images_count = sum([len(os.listdir(os.path.join(test_dir, class_name))) for class_name in os.listdir(test_dir)])

# Wyświetlanie wyników
print("Liczba zdjęć w katalogu treningowym: ", train_images_count)
print("Liczba zdjęć w katalogu walidacyjnym: ", val_images_count)
print("Liczba zdjęć w katalogu testowym: ", test_images_count)


Liczba zdjęć w katalogu treningowym:  14338
Liczba zdjęć w katalogu walidacyjnym:  3097
Liczba zdjęć w katalogu testowym:  3145


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Ścieżki do katalogów
train_dir = './data/train'
val_dir = './data/val'
test_dir = './data/test'

# Stworzenie generatorów
# 1. Generator dla danych treningowych z augmentacją
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalizacja pikseli do zakresu [0, 1]
    rotation_range=40,  # Przypadkowe obracanie obrazów
    width_shift_range=0.2,  # Przesunięcie obrazu w poziomie
    height_shift_range=0.2,  # Przesunięcie obrazu w pionie
    shear_range=0.2,  # Zniekształcenie obrazu
    zoom_range=0.2,  # Powiększenie obrazu
    horizontal_flip=True,  # Odbicie poziome
    fill_mode='nearest'  # Wypełnianie nowych pikseli
)

# 2. Generator dla danych walidacyjnych i testowych (bez augmentacji)
# Tylko normalizacja pikseli
test_datagen = ImageDataGenerator(rescale=1./255)

# 3. Tworzenie generatorów danych z katalogów
train_generator = train_datagen.flow_from_directory(
    train_dir,  # Katalog z danymi treningowymi
    target_size=(150, 150),  # Rozmiar obrazu, do którego zostaną dopasowane wszystkie obrazy
    batch_size=32,  # Liczba obrazów w jednej paczce
    class_mode='categorical'  # Typ etykiety (wieloklasowa klasyfikacja)
)

validation_generator = test_datagen.flow_from_directory(
    val_dir,  # Katalog z danymi walidacyjnymi
    target_size=(150, 150),  # Rozmiar obrazu
    batch_size=32,  # Liczba obrazów w jednej paczce
    class_mode='categorical'  # Typ etykiety
)

test_generator = test_datagen.flow_from_directory(
    test_dir,  # Katalog z danymi testowymi
    target_size=(150, 150),  # Rozmiar obrazu
    batch_size=32,  # Liczba obrazów w jednej paczce
    class_mode='categorical'  # Typ etykiety
)

# Opcjonalnie możesz sprawdzić pierwsze obrazy w generatorze
for data_batch, labels_batch in train_generator:
    print(data_batch.shape)  # Rozmiar paczki obrazów
    print(labels_batch.shape)  # Rozmiar paczki etykiet
    break


Found 14338 images belonging to 120 classes.
Found 3097 images belonging to 120 classes.
Found 3145 images belonging to 120 classes.
(32, 150, 150, 3)
(32, 120)


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
# Załaduj model ResNet50 bez wag, aby wykorzystać go jako bazę
base_model = ResNet50(weights=None, include_top=False, input_shape=(150, 150, 3))

# Dodajemy własne warstwy na końcu sieci
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Global Average Pooling, aby zmniejszyć wymiary
x = Dense(512, activation='relu')(x)  # Gęsta warstwa
predictions = Dense(120, activation='softmax')(x)  # Ostateczna warstwa klasyfikacji (120 klas)

# Tworzymy model
model = Model(inputs=base_model.input, outputs=predictions)

# 2. Zamrożenie wszystkich warstw poza ostatnimi 10, aby dostroić model
for layer in base_model.layers[:-10]:  # Zamrażamy wszystkie warstwy oprócz ostatnich 10
    layer.trainable = False


In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),  # Używamy Adam jako optymalizator
              loss='categorical_crossentropy',  # Funkcja straty do wieloklasowej klasyfikacji
              metrics=['accuracy'])  # Metryka do oceny wyników


In [ ]:
history = model.fit(
    train_generator,  # Generator dla danych treningowych
    steps_per_epoch=train_generator.samples // train_generator.batch_size,  # Liczba kroków na epokę
    epochs=20,  # Liczba epok
    validation_data=validation_generator,  # Generator dla danych walidacyjnych
    validation_steps=validation_generator.samples // validation_generator.batch_size,  # Liczba kroków na epokę walidacyjną
)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 15/448 ━━━━━━━━━━━━━━━━━━━━ 20:48 3s/step - accuracy: 0.0083 - loss: 4.7907